# Setup Enviroment

In [ ]:
import os

# Get the name of the current directory
current_dir = os.path.basename(os.getcwd())

# Check for the two possible names
if current_dir.lower() in {"notebooks", "notebook"}:
    # Move one level up
    os.chdir("..")

# Install Dependencies

In [1]:
!pip install -q -U pip
!pip install -q coqui-tts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 129.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 116.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 126.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 100.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.8/861.8 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

# Import Libraries

In [3]:
import os
import glob
import json
import pandas as pd
import soundfile as sf
from datasets import load_dataset
from sklearn.model_selection import train_test_split
# TTS specific imports
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from trainer import Trainer, TrainerArgs

2025-05-06 04:27:33.974663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746505654.152001      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746505654.204447      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Python 3.11.11

# numpy==1.26.4
# json: 2.0.9
# pandas: 2.2.3
# soundfile: 0.13.1
# datasets: 3.5.0
# TTS: 0.26.0
# trainer: 0.2.3


# Dataset 

In [ ]:
!wget -q https://huggingface.co/datasets/ntt123/viet-tts-dataset/resolve/main/viet-tts.tar.gz -O viet-tts.tar.gz
!mkdir data
!tar -C data -xzf viet-tts.tar.gz
!rm -rf viet-tts.tar.gz

In [ ]:
df = pd.read_csv("data/meta_data.tsv", delimiter="\t")
df

,wav/000000.wav,Ai đây tức là một kẻ ăn mày vậy. Anh ta chưa kịp quay đi thì đã thấy mấy con chó vàng chạy xồng xộc ra cứ nhảy xổ vào chân anh.
0,wav/000001.wav,Anh mau cứu tôi! những lời nói run sợ một cách...
1,wav/000002.wav,"Anh đi đường cái quan đi ba bước rồi dừng lại,..."
2,wav/000003.wav,Anh đi đường cái quan đáo để ấy mỉm cười đắc c...
3,wav/000004.wav,Biết thế thì thịt quách gà đi lại được lợi cái...
4,wav/000005.wav,Cho vui! Phúc bĩu môi nghĩ thầm. Anh muốn đuổi...
...,...,...
22878,wav/022879.wav,"Ừ, trả thù, và trả thù!"
22879,wav/022880.wav,"Ừ, tôi vẫn biết là mợ đã chán tôi lắm, mợ còn ..."
22880,wav/022881.wav,"Ừ, việc ấy là như thế, thì nào Dung có làm gì ..."
22881,wav/022882.wav,"Ừ, được rồi!..."


In [ ]:
# Define paths
output_dataset = "data"

# Define the custom formatter
def my_custom_format(dataset_path, meta_file, **kwargs):
    samples = []
    metadata_path = os.path.join(dataset_path, meta_file)
    
    with open(metadata_path, "r", encoding="utf-8") as f:
        for line in f:
            file_name, transcription = line.strip().split("\t")
            audio_path = os.path.join(dataset_path, file_name)
            samples.append({
                "audio_file": audio_path,
                "text": transcription,
                "speaker_name": None,
                "language": "vi",
                "audio_unique_name": file_name
            })
    return samples

# Define dataset config
dataset_config = BaseDatasetConfig(
    formatter="my_custom_format",
    meta_file_train="meta_data.tsv",
    path=output_dataset
)

In [7]:
def get_vietnamese_chars():
    return (
        "aàáảãạăằắẳẵặâầấẩẫậbcdfđeèéẻẽẹêềếểễệghiìíỉĩịjklmnoòóỏõọôồốổỗộơờớởỡợpqrstuùúủũụưừứửữựvwxyỳýỷỹỵz"
        + "AÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬBCDFĐEÈÉẺẼẸÊỀẾỂỄỆGHIÌÍỈĨỊJKLMNOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢPQRSTUÙÚỨỬỮỰVWXYỲÝỶỸỴZ"
        + "0123456789"
    )

def get_characters_config():
    return CharactersConfig(
        pad="<PAD>",
        eos="<EOS>",
        bos="<BOS>",
        blank=None,
        characters=get_vietnamese_chars(),
        punctuations=".,!? ",
        phonemes=None,
        characters_class="TTS.tts.models.vits.VitsCharacters"
    )

def get_glow_tts_config(output_path="./ckpts", config_path=None):
    characters_config = get_characters_config()
    
    if config_path and os.path.exists(config_path):
        print("Load Config from existed directory")
        with open(config_path, "r") as f:
            config_dict = json.load(f)
        valid_keys = GlowTTSConfig.__init__.__code__.co_varnames
        filtered_config_dict = {k: v for k, v in config_dict.items() if k in valid_keys}
        config = GlowTTSConfig(**filtered_config_dict)
        config.characters = characters_config
        
        config.output_path = output_path
    else:
        print("Create new Config")
        config = GlowTTSConfig(
            batch_size=64,
            eval_batch_size=64,
            num_loader_workers=4,
            num_eval_loader_workers=4,
            run_eval=True,
            test_delay_epochs=-1,
            epochs=100,
            text_cleaner="multilingual_cleaners",
            use_phonemes=False,
            phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
            print_step=300,
            print_eval=False,
            mixed_precision=True,
            output_path=output_path,
            save_step=1000,
            eval_split_max_size=256,
            eval_split_size=0.1,
            characters=characters_config,
            save_n_checkpoints=1,
            test_sentences = ["Tôi đã mất khá nhiều thời gian để phát triển một giọng nói, và giờ đây khi đã có nó, tôi sẽ không im lặng.",
                       "Hãy là một giọng nói, không phải tiếng vọng.",
                       "Xin lỗi Dave. Tôi e là tôi không thể làm điều đó.",
                       "Chiếc bánh này tuyệt vời. Nó thật ngon và ẩm.",
                       "Trước ngày hai mươi hai tháng mười một, năm một nghìn chín trăm sáu mươi ba."]
        )
    
    return config

# Usage
output_path = "./ckpts"
run_folder_path = "run-May-02-2025_07+04PM-0000000"
config_path = os.path.join(output_path, run_folder_path, "config.json")
config = get_glow_tts_config(output_path, config_path)

Load Config from existed directory


In [8]:
# Initialize AudioProcessor and Tokenizer
ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)

In [9]:
# Load dataset samples
def load_custom_tts_samples(dataset_config):
    samples = my_custom_format(
        dataset_path=dataset_config.path,
        meta_file=dataset_config.meta_file_train
    )
    eval_split_size = config.eval_split_size
    
    # Use train_test_split to split samples
    train_samples, eval_samples = train_test_split(
        samples,
        test_size=eval_split_size,
        random_state=42
    )
    
    return train_samples, eval_samples

train_samples, eval_samples = load_custom_tts_samples(dataset_config)

In [10]:
from IPython.display import Audio

for dataset, name in [(train_samples, "train sample"), (eval_samples, "eval sample")]:
    sample = dataset[99]
    print(f"{name.capitalize()}:")
    print(sample['text'])
    display(Audio(sample['audio_file']))

Train sample:
- Trong lưng giắt đến ba, bốn chục bạc từ Phủ Lý ra đây định cân sâm cho con giai đang ốm thập tử nhất sinh đấy...


Eval sample:
Bọn này thuộc vào lớp người mà do ai, xã hội được hoạt động, trông vào đâu, cũng thấy danh lợi, sống lên trên những cuộc tai họa hoặc những sự may mắn của người đời.


In [10]:
# Defind Model
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [11]:
# Find the latest checkpoint
checkpoint_dir = os.path.join(output_path, run_folder_path)
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "checkpoint_*.pth"))
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model.load_checkpoint(config, latest_checkpoint, eval=False)
else:
    print("No checkpoint found, starting training from scratch.")


Resuming training from checkpoint: ./ckpts/run-May-02-2025_07+04PM-0000000/checkpoint_32000.pth


In [12]:
# Initialize Trainer
trainer = Trainer(
    TrainerArgs(
        continue_path=checkpoint_dir if checkpoint_files else None,  # Resume from checkpoint directory
        restore_path=latest_checkpoint if checkpoint_files else None  # Restore model state
    ),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 2
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=./ckpts/run-May-02-2025_07+04PM-0000000
 > Restoring from checkpoint_32000.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 32000

 > Model has 28623121 parameters


In [ ]:
# --> EVAL PERFORMANCE
#      | > avg_loader_time: 0.6213016510009766 (+0.03184342384338379)
#      | > avg_loss: -0.9545338119779314 (-0.010085366453443267)
#      | > avg_log_mle: -1.0004474197115216 (-0.010152942793709818)
#      | > avg_loss_dur: 0.045913600495883396 (+6.757155060767989e-05)

In [13]:
trainer.fit()

 > Restoring best loss from best_model_31556.pth ...
 > Starting with loaded last best loss {'train_loss': -0.9507075430916958, 'eval_loss': None}

 > EPOCH: 0/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 04:31:55) 

   --> TIME: 2025-05-06 04:32:47 -- STEP: 99/322 -- GLOBAL_STEP: 32100
     | > loss: -1.0038946866989136  (-0.9749497127051305)
     | > log_mle: -1.0516550540924072  (-1.0211305871154326)
     | > loss_dur: 0.047760359942913055  (0.046180873055650745)
     | > amp_scaler: 1024.0  (1024.0)
     | > grad_norm: tensor(20.8182, device='cuda:0')  (tensor(39.7450, device='cuda:0'))
     | > current_lr: 2.5e-07 
     | > step_time: 0.4918  (0.4471461628422593)
     | > loader_time: 0.0138  (0.007111368757305723)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6372686181749616 (+0.0)
     | > avg_loss: -0.9277584331376212 (+0.0)
     | > avg_log_mle: -0.973495364189148 (+0.0)
     | > avg_loss_dur: 0.04573693158371108 (+0.0)

 > BEST MODEL : ckpts/run-May-02-2025_07+04PM-0000000/best_model_32323.pth

 > EPOCH: 1/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 04:38:31) 

   --> TIME: 2025-05-06 04:39:05 -- STEP: 77/322 -- GLOBAL_STEP: 32400
     | > loss: -0.9373217821121216  (-0.8647424594148413)
     | > log_mle: -0.9850516319274902  (-0.9124196210464874)
     | > loss_dur: 0.04772982373833656  (0.04767716463123049)
     | > amp_scaler: 512.0  (518.6493506493506)
     | > grad_norm: tensor(135.6389, device='cuda:0')  (tensor(224.8216, device='cuda:0'))
     | > current_lr: 2.4999999999999998e-05 
     | > step_time: 0.5019  (0.4130130371490082)
     | > loader_time: 0.009  (0.006780060854825107)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6550652980804443 (+0.017796679905482726)
     | > avg_loss: -0.9417918017932347 (-0.014033368655613554)
     | > avg_log_mle: -0.986799168586731 (-0.013303804397583052)
     | > avg_loss_dur: 0.04500737264752388 (-0.0007295589361872024)

 > BEST MODEL : ckpts/run-May-02-2025_07+04PM-0000000/best_model_32645.pth

 > EPOCH: 2/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 04:44:53) 

   --> TIME: 2025-05-06 04:45:16 -- STEP: 55/322 -- GLOBAL_STEP: 32700
     | > loss: -0.9589123129844666  (-0.9527878046035767)
     | > log_mle: -1.003960371017456  (-0.9981031591242009)
     | > loss_dur: 0.04504803940653801  (0.04531534903428771)
     | > amp_scaler: 512.0  (512.0)
     | > grad_norm: tensor(166.9768, device='cuda:0')  (tensor(174.1974, device='cuda:0'))
     | > current_lr: 2.525e-05 
     | > step_time: 0.4093  (0.3745477329600941)
     | > loader_time: 0.0115  (0.005505782907659358)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6327776772635324 (-0.022287620816911935)
     | > avg_loss: -0.9240378788539342 (+0.01775392293930056)
     | > avg_log_mle: -0.9687587193080357 (+0.018040449278695303)
     | > avg_loss_dur: 0.044720837793179924 (-0.0002865348543439547)


 > EPOCH: 3/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 04:51:09) 

   --> TIME: 2025-05-06 04:51:22 -- STEP: 33/322 -- GLOBAL_STEP: 33000
     | > loss: -0.9259595274925232  (-0.9608791055101337)
     | > log_mle: -0.9720196723937988  (-1.0058159394697705)
     | > loss_dur: 0.04606013372540474  (0.0449368337338621)
     | > amp_scaler: 512.0  (512.0)
     | > grad_norm: tensor(124.9697, device='cuda:0')  (tensor(161.7702, device='cuda:0'))
     | > current_lr: 2.55e-05 
     | > step_time: 0.3543  (0.33402336727489124)
     | > loader_time: 0.0053  (0.004969676335652669)


 > CHECKPOINT : ckpts/run-May-02-2025_07+04PM-0000000/checkpoint_33000.pth



 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6289747646876744 (+0.02736455372401647)
     | > avg_loss: -1.097258196558271 (+0.007664026532854473)
     | > avg_log_mle: -1.1411592040743144 (+0.007321507590158038)
     | > avg_loss_dur: 0.04390099335994039 (+0.0003425011677401363)


 > EPOCH: 97/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 14:17:09) 

   --> TIME: 2025-05-06 14:17:36 -- STEP: 65/322 -- GLOBAL_STEP: 63300
     | > loss: -1.1423840522766113  (-1.1558613758820755)
     | > log_mle: -1.178001046180725  (-1.190297792508052)
     | > loss_dur: 0.03561696037650108  (0.03443641370305647)
     | > amp_scaler: 1024.0  (1024.0)
     | > grad_norm: tensor(100.1049, device='cuda:0')  (tensor(100.9727, device='cuda:0'))
     | > current_lr: 4.9e-05 
     | > step_time: 0.5211  (0.38863581877488357)
     | > loader_time: 0.0074  (0.006096076965332031)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6250875200544086 (-0.0038872446332658406)
     | > avg_loss: -1.0883476461683002 (+0.008910550389970906)
     | > avg_log_mle: -1.130781926427569 (+0.010377277646745453)
     | > avg_loss_dur: 0.04243428143007415 (-0.001466711929866242)


 > EPOCH: 98/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 14:23:17) 

   --> TIME: 2025-05-06 14:23:33 -- STEP: 43/322 -- GLOBAL_STEP: 63600
     | > loss: -1.1114286184310913  (-1.1603197025698284)
     | > log_mle: -1.1410106420516968  (-1.1932504260262775)
     | > loss_dur: 0.029582081362605095  (0.03293072640202767)
     | > amp_scaler: 1024.0  (1024.0)
     | > grad_norm: tensor(85.7976, device='cuda:0')  (tensor(114.7148, device='cuda:0'))
     | > current_lr: 4.925e-05 
     | > step_time: 0.3796  (0.347862859104955)
     | > loader_time: 0.0053  (0.005383474882258925)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6160542283739362 (-0.0090332916804724)
     | > avg_loss: -1.1219335181372507 (-0.033585871968950576)
     | > avg_log_mle: -1.1645064932959421 (-0.03372456686837322)
     | > avg_loss_dur: 0.04257296760167394 (+0.00013868617159979585)


 > EPOCH: 99/100
 --> ckpts/run-May-02-2025_07+04PM-0000000

 > TRAINING (2025-05-06 14:29:27) 

   --> TIME: 2025-05-06 14:29:35 -- STEP: 21/322 -- GLOBAL_STEP: 63900
     | > loss: -1.2160719633102417  (-1.1870399940581549)
     | > log_mle: -1.2480295896530151  (-1.2181699219204134)
     | > loss_dur: 0.03195766359567642  (0.031129922274322736)
     | > amp_scaler: 1024.0  (1024.0)
     | > grad_norm: tensor(135.0472, device='cuda:0')  (tensor(120.0423, device='cuda:0'))
     | > current_lr: 4.95e-05 
     | > step_time: 0.3248  (0.31408123742966426)
     | > loader_time: 0.0044  (0.004940714154924665)


 > CHECKPOINT : ckpts/run-May-02-2025_07+04PM-0000000/checkpoint_64000.pth



   --> TIME: 2025-05-06 14:34:50 -- STEP: 321/322 -- GLOBAL_STEP: 64200
     | > loss: -1.139192819595337  (-1.1461634561651597)
     | > log_mle: -1.1896342039108276  (-1.1868862626708554)
     | > loss_dur: 0.05044136941432953  (0.040722804967998696)
     | > amp_scaler: 1024.0  (1024.0)
     | > grad_norm: tensor(93.2843, device='cuda:0')  (tensor(114.0992, device='cuda:0'))
     | > current_lr: 4.95e-05 
     | > step_time: 1.5738  (0.7975553745792661)
     | > loader_time: 0.0148  (0.18739397280684145)


 > EVALUATION 




  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6140463147844587 (-0.0020079135894774947)
     | > avg_loss: -1.1667222261428833 (-0.044788708005632616)
     | > avg_log_mle: -1.2084908519472395 (-0.04398435865129735)
     | > avg_loss_dur: 0.04176862952964647 (-0.0008043380720274759)

 > BEST MODEL : ckpts/run-May-02-2025_07+04PM-0000000/best_model_64201.pth


In [14]:
!echo abc

abc


# Save and Load Model

## Save Model

In [35]:
import os
import torch
import json
from pathlib import Path
from TTS.tts.configs.shared_configs import CharactersConfig

def save_model_elements(model, config, tokenizer, ap, save_dir):
    """
    Save all elements of the GlowTTS model, including state, config, tokenizer, and audio processor.
    
    Args:
        model: The GlowTTS model instance.
        config: The GlowTTSConfig configuration object.
        tokenizer: The TTSTokenizer instance.
        ap: The AudioProcessor instance.
        save_dir: Directory to save the model elements.
    """
    # Create save directory if it doesn't exist
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # 1. Save model state dictionary
    model_path = save_dir / "model_state.pth"
    checkpoint = {
        "model": model.state_dict(),  # Wrap state_dict in 'model' key for TTS.api
        "step": getattr(model, "global_step", 32000),  # From training log
    }
    torch.save(checkpoint, model_path)
    print(f"Model state saved to {model_path}")
    
    # 2. Save configuration
    config_path = save_dir / "config.json"
    config_dict = {
        "batch_size": config.batch_size,
        "eval_batch_size": config.eval_batch_size,
        "num_loader_workers": config.num_loader_workers,
        "num_eval_loader_workers": config.num_eval_loader_workers,
        "run_eval": config.run_eval,
        "test_delay_epochs": config.test_delay_epochs,
        "epochs": config.epochs,
        "text_cleaner": config.text_cleaner,
        "use_phonemes": config.use_phonemes,
        "phoneme_cache_path": config.phoneme_cache_path,
        "print_step": config.print_step,
        "print_eval": config.print_eval,
        "mixed_precision": config.mixed_precision,
        "output_path": config.output_path,
        "save_step": config.save_step,
        "eval_split_max_size": config.eval_split_max_size,
        "eval_split_size": config.eval_split_size,
        "save_n_checkpoints": config.save_n_checkpoints,
        "test_sentences": config.test_sentences,
        "characters": {
            "pad": config.characters.pad,
            "eos": config.characters.eos,
            "bos": config.characters.bos,
            "blank": config.characters.blank,
            "characters": config.characters.characters,
            "punctuations": config.characters.punctuations,
            "phonemes": getattr(config.characters, "phonemes", None),
            "characters_class": config.characters.characters_class
        }
    }
    with open(config_path, "w", encoding="utf-8") as f:
        json.dump(config_dict, f, ensure_ascii=False, indent=2)
    print(f"Configuration saved to {config_path}")
    
    # 3. Save tokenizer
    tokenizer_path = save_dir / "tokenizer.json"
    tokenizer_config = {
        "characters": {
            "pad": tokenizer.characters.pad,
            "eos": tokenizer.characters.eos,
            "bos": tokenizer.characters.bos,
            "blank": tokenizer.characters.blank,
            "characters": tokenizer.characters.characters,
            "punctuations": tokenizer.characters.punctuations,
            "phonemes": getattr(tokenizer.characters, "phonemes", None)
        },
        "pad_token_id": getattr(tokenizer, "pad_token_id", None),
        "eos_token_id": getattr(tokenizer, "eos_token_id", None),
        "bos_token_id": getattr(tokenizer, "bos_token_id", None)
    }
    with open(tokenizer_path, "w", encoding="utf-8") as f:
        json.dump(tokenizer_config, f, ensure_ascii=False, indent=2)
    print(f"Tokenizer saved to {tokenizer_path}")
    
    # 4. Save audio processor settings
    ap_path = save_dir / "audio_processor.json"
    ap_config = {
        "sample_rate": ap.sample_rate,
        "num_mels": ap.num_mels,
        "min_level_db": getattr(ap, "min_level_db", -100),
        "frame_length_ms": getattr(ap, "frame_length_ms", 50),
        "frame_shift_ms": getattr(ap, "frame_shift_ms", 12.5),
        "preemphasis": getattr(ap, "preemphasis", 0.97),
        "ref_level_db": getattr(ap, "ref_level_db", 20),
        "power": getattr(ap, "power", 1.5),
        "griffin_lim_iters": getattr(ap, "griffin_lim_iters", 60),
        "n_fft": getattr(ap, "n_fft", 1024),
        "hop_length": getattr(ap, "hop_length", 256),
        "win_length": getattr(ap, "win_length", 1024)
    }
    with open(ap_path, "w", encoding="utf-8") as f:
        json.dump(ap_config, f, ensure_ascii=False, indent=2)
    print(f"Audio processor settings saved to {ap_path}")

# Usage
output_path = "./ckpts"
run_folder_path = "run-May-02-2025_07+04PM-0000000"
save_directory = os.path.join(output_path, run_folder_path, "model_elements")
save_model_elements(model, config, tokenizer, ap, save_directory)

Model state saved to viet-fine-tune-glow-tts/run-May-02-2025_07+04PM-0000000/model_elements/model_state.pth
Configuration saved to ckpts/run-May-02-2025_07+04PM-0000000/model_elements/config.json
Tokenizer saved to ckpts/run-May-02-2025_07+04PM-0000000/model_elements/tokenizer.json
Audio processor settings saved to ckpts/run-May-02-2025_07+04PM-0000000/model_elements/audio_processor.json


## Load Model

In [36]:
import os
import torch
import json
from pathlib import Path
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from TTS.tts.configs.shared_configs import CharactersConfig

def load_model_elements(load_dir, device="cpu"):
    """
    Load all elements of the GlowTTS model, including state, config, tokenizer, and audio processor.
    
    Args:
        load_dir: Directory where model elements are saved.
        device: Device to load the model onto ('cpu' or 'cuda').
    
    Returns:
        model: Loaded GlowTTS model.
        config: Loaded GlowTTSConfig.
        tokenizer: Loaded TTSTokenizer.
        ap: Loaded AudioProcessor.
    """
    load_dir = Path(load_dir)
    
    # 1. Load configuration
    config_path = load_dir / "config.json"
    if not config_path.exists():
        raise FileNotFoundError(f"Config file not found at {config_path}")
    with open(config_path, "r", encoding="utf-8") as f:
        config_dict = json.load(f)
    
    # Reconstruct CharactersConfig
    characters_config = CharactersConfig(**config_dict["characters"])
    config_dict["characters"] = characters_config
    
    # Initialize GlowTTSConfig
    config = GlowTTSConfig(**{k: v for k, v in config_dict.items() if k != "characters"})
    config.characters = characters_config
    print(f"Configuration loaded from {config_path}")
    
    # 2. Load tokenizer
    tokenizer_path = load_dir / "tokenizer.json"
    if not tokenizer_path.exists():
        raise FileNotFoundError(f"Tokenizer file not found at {tokenizer_path}")
    with open(tokenizer_path, "r", encoding="utf-8") as f:
        tokenizer_config = json.load(f)
    
    # Initialize tokenizer
    tokenizer, _ = TTSTokenizer.init_from_config(config)
    print(f"Tokenizer loaded from {tokenizer_path}")
    
    # 3. Load audio processor
    ap_path = load_dir / "audio_processor.json"
    if not ap_path.exists():
        raise FileNotFoundError(f"Audio processor file not found at {ap_path}")
    with open(ap_path, "r", encoding="utf-8") as f:
        ap_config = json.load(f)
    
    # Initialize AudioProcessor and apply saved settings
    ap = AudioProcessor.init_from_config(config)
    for key, value in ap_config.items():
        setattr(ap, key, value)
    print(f"Audio processor loaded from {ap_path}")
    
    # 4. Initialize model
    model = GlowTTS(config, ap, tokenizer, speaker_manager=None)
    
    # 5. Load model state
    model_path = load_dir / "model_state.pth"
    if not model_path.exists():
        raise FileNotFoundError(f"Model state file not found at {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model"])
    model.to(device)
    model.eval()
    print(f"Model state loaded from {model_path}")
    
    return model, config, tokenizer, ap

## Inference from Load elements

In [37]:
import torch
import IPython

def infer_with_glow_tts(model, ap, tokenizer, text, output_wav_path="output.wav"):
    """
    Perform inference with the GlowTTS model to generate speech from text.
    
    Args:
        model: Loaded GlowTTS model.
        ap: Loaded AudioProcessor.
        tokenizer: Loaded TTSTokenizer.
        text: Input text to synthesize.
        output_wav_path: Path to save the generated WAV file.
    """
    # Tokenize input text
    text_inputs = tokenizer.text_to_ids(text)
    text_inputs = torch.LongTensor(text_inputs).unsqueeze(0).to(model.device)  # Shape: [1, seq_len]
    
    # Compute sequence lengths
    x_lengths = torch.LongTensor([text_inputs.size(1)]).to(model.device)  # Shape: [1]
    
    # Perform inference with aux_input
    with torch.no_grad():
        outputs = model.inference(x=text_inputs, aux_input={"x_lengths": x_lengths})
    
    # Extract mel-spectrogram
    mel = outputs["model_outputs"]
    if mel.dim() == 3:
        mel = mel.squeeze(0)
    
    # Convert mel-spectrogram to audio using Griffin-Lim
    audio = ap.griffin_lim(mel.T.cpu().numpy())
    
    # Save audio to WAV file
    ap.save_wav(audio, output_wav_path)
    print(f"Generated audio saved to {output_wav_path}")

# Usage
load_directory = os.path.join("./ckpts", "run-May-02-2025_07+04PM-0000000", "model_elements")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and components
model, config, tokenizer, ap = load_model_elements(load_directory, device=device)

# Perform inference
text = "Trong khi đó, tại bến tàu du lịch Nha Trang, hàng ngàn du khách chen nhau để đi đến các đảo trên vịnh Nha Trang, lực lượng cảnh sát đường thủy đã tăng cường quân số để quản lý, đảm bảo an toàn cho du khách."
out_path = "abc.wav"
infer_with_glow_tts(model, ap, tokenizer, text, out_path)

# Play audio
IPython.display.Audio(out_path)

Configuration loaded from ckpts/run-May-02-2025_07+04PM-0000000/model_elements/config.json
Tokenizer loaded from ckpts/run-May-02-2025_07+04PM-0000000/model_elements/tokenizer.json
Audio processor loaded from ckpts/run-May-02-2025_07+04PM-0000000/model_elements/audio_processor.json


/tmp/ipykernel_31/4046368212.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


Model state loaded from ckpts/run-May-02-2025_07+04PM-0000000/model_elements/model_state.pth


AttributeError: 'AudioProcessor' object has no attribute 'griffin_lim'

In [ ]:
%cd /kaggle/working/ckpts

In [ ]:
!rm -rf tokenizer.pkl model_checkpoint.pth config.json audio_processor.pkl

In [ ]:
!echo done

# Inference

In [ ]:
!echo not_done

In [15]:
def get_vietnamese_chars():
    return (
        "aàáảãạăằắẳẵặâầấẩẫậbcdfđeèéẻẽẹêềếểễệghiìíỉĩịjklmnoòóỏõọôồốổỗộơờớởỡợpqrstuùúủũụưừứửữựvwxyỳýỷỹỵz"
        + "AÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬBCDFĐEÈÉẺẼẸÊỀẾỂỄỆGHIÌÍỈĨỊJKLMNOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢPQRSTUÙÚỨỬỮỰVWXYỲÝỶỸỴZ"
        + "0123456789"
    )

def get_characters_config():
    return CharactersConfig(
        pad="<PAD>",
        eos="<EOS>",
        bos="<BOS>",
        blank=None,
        characters=get_vietnamese_chars(),
        punctuations=".,!? ",
        phonemes=None,
        characters_class="TTS.tts.models.vits.VitsCharacters"
    )

def get_glow_tts_config(output_path="./ckpts", config_path=None):
    characters_config = get_characters_config()
    
    if config_path and os.path.exists(config_path):
        print("Load Config from existed directory")
        with open(config_path, "r") as f:
            config_dict = json.load(f)
        valid_keys = GlowTTSConfig.__init__.__code__.co_varnames
        filtered_config_dict = {k: v for k, v in config_dict.items() if k in valid_keys}
        config = GlowTTSConfig(**filtered_config_dict)
        config.characters = characters_config
        
        config.output_path = output_path
    else:
        print("Create new Config")
        config = GlowTTSConfig(
            batch_size=64,
            eval_batch_size=64,
            num_loader_workers=4,
            num_eval_loader_workers=4,
            run_eval=True,
            test_delay_epochs=-1,
            epochs=100,
            text_cleaner="multilingual_cleaners",
            use_phonemes=False,
            phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
            print_step=300,
            print_eval=False,
            mixed_precision=True,
            output_path=output_path,
            save_step=1000,
            eval_split_max_size=256,
            eval_split_size=0.1,
            characters=characters_config,
            save_n_checkpoints=1,
            test_sentences = ["Tôi đã mất khá nhiều thời gian để phát triển một giọng nói, và giờ đây khi đã có nó, tôi sẽ không im lặng.",
                       "Hãy là một giọng nói, không phải tiếng vọng.",
                       "Xin lỗi Dave. Tôi e là tôi không thể làm điều đó.",
                       "Chiếc bánh này tuyệt vời. Nó thật ngon và ẩm.",
                       "Trước ngày hai mươi hai tháng mười một, năm một nghìn chín trăm sáu mươi ba."]
        )
    
    return config

# Usage
output_path = "./ckpts"
run_folder_path = "run-May-02-2025_07+04PM-0000000"
config_path = os.path.join(output_path, run_folder_path, "config.json")
config = get_glow_tts_config(output_path, config_path)

# Initialize AudioProcessor and Tokenizer
ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)

# Defind Model
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# Find the latest checkpoint
checkpoint_dir = os.path.join(output_path, run_folder_path)
checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "best_model.pth"))
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model.load_checkpoint(config, latest_checkpoint, eval=False)
else:
    print("No checkpoint found, starting training from scratch.")

# Initialize Trainer
trainer = Trainer(
    TrainerArgs(
        continue_path=checkpoint_dir if checkpoint_files else None,  # Resume from checkpoint directory
        restore_path=latest_checkpoint if checkpoint_files else None  # Restore model state
    ),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)



Load Config from existed directory
Resuming training from checkpoint: ./ckpts/run-May-02-2025_07+04PM-0000000/best_model.pth


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 2
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=./ckpts/run-May-02-2025_07+04PM-0000000
 > Restoring from checkpoint_32000.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 32000

 > Model has 28623121 parameters


# Good Here

In [15]:
from TTS.api import TTS
import torch

# Define paths and input
check_point_folder = "ckpts/run-May-02-2025_07+04PM-0000000"
model_path = f"{check_point_folder}/best_model.pth"
config_path = f"{check_point_folder}/config.json"

# model_path = f"/kaggle/working/ckpts/model_checkpoint.pth"
# config_path = f"/kaggle/working/ckpts/config.json"

out_path = "bca.wav"
text = "Trong khi đó, tại bến tàu du lịch Nha Trang, hàng ngàn du khách chen nhau để đi đến các đảo trên vịnh Nha Trang, lực lượng cảnh sát đường thủy đã tăng cường quân số để quản lý, đảm bảo an toàn cho du khách."

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize TTS model
tts = TTS(model_path=model_path, config_path=config_path, progress_bar=True)

# Move model to the specified device
tts.to(device)

# Perform inference and save to file
tts.tts_to_file(text=text, file_path=out_path, speaker=None, language=None, split_sentences=False)

'bca.wav'

In [16]:
import IPython
IPython.display.Audio("bca.wav")

In [17]:
!rm -rf /kaggle/working/dataset

In [18]:
!cp -r ./ckpts/run-May-02-2025_07+04PM-0000000/best_model.pth ./ckpts/
!cp -r ./ckpts/run-May-02-2025_07+04PM-0000000/config.json ./ckpts/


In [18]:
!mv asr_outputwav demo_output/asr_output.wav

In [20]:
%cd ..

/kaggle/working


In [ ]:
from huggingface_hub import HfApi, login

# Log in
login(token="<your_hf_token>")

# Upload a single file
api = HfApi()
# api.upload_file(
#     path_or_fileobj="/path/to/local/file.txt",
#     path_in_repo="file.txt",
#     repo_id="your-username/your-space-name",
#     repo_type="space"
# )

# Upload an entire folder
api.upload_folder(
    folder_path="ckpts",
    repo_id="danhtran2mind/Viet-Glow-TTS-finetuning",
    repo_type="model"
)

best_model_64201.pth:   0%|          | 0.00/344M [00:00<?, ?B/s]

checkpoint_64000.pth:   0%|          | 0.00/344M [00:00<?, ?B/s]

best_model_64201.pth:   0%|          | 0.00/344M [00:00<?, ?B/s]

best_model_64201.pth:   0%|          | 0.00/344M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1746505880.6523aa8279ee.31.0:   0%|          | 0.00/256M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/danhtran2mind/Viet-Glow-TTS-finetuning/commit/0a57cee67bf462ae5a5bd94e77e8397b4da487c2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0a57cee67bf462ae5a5bd94e77e8397b4da487c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/danhtran2mind/ckpts', endpoint='https://huggingface.co', repo_type='model', repo_id='danhtran2mind/ckpts'), pr_revision=None, pr_num=None)

## Evaluation Step

# TensorflowBoard Plotting

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir